In [1]:
%load_ext autoreload
%autoreload 2

#per la riproducibilità
%env CUBLAS_WORKSPACE_CONFIG=":4096:8"

import os
import itertools
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from IPython.display import Markdown as md
from multiprocessing import Pool
import time
from tqdm import tqdm
import networkx as nx
from networkx import stochastic_block_model


from graph_generation import GenerateGraph, perturb_nx_graph
from models import GCN, view_parameters, new_parameters, modify_parameters, Inits, new_parameters_linears
from train import Trainer, Dataset
from embedding import Embedding
from config_valid import Config, TrainingMode
import experiments
from experiments import Experiments, experiment_node_embedding, all_seeds
from plot_funcs import (plot_dim1, plot_dimN, plot_correlation_error, plot_metrics, plot_node_emb_1D, plot_node_emb_nD, scatter_node_emb, 
                        plot_graph_emb_1D, plot_graph_emb_nD, plot_data_degree_sequence, plot_corr_epoch, plot_ripetizioni_stesso_trial, 
                        plot_onlyloss_ripetizioni_stesso_trial,plot_onlyloss_ripetizioni_stesso_trial_superimposed, Data2Plot, plot_weights_multiple_hist)
from plot_model import plot_model
from utils import array_wo_outliers, plot_grafo, plot_grafo2
from Inspect import Inspect


import torch
from torch_geometric import nn
from torch_geometric.loader import DataLoader
device = torch.device('cuda')

from scipy import stats
from sklearn.metrics import log_loss

import pickle 
from plt_parameters import init_params, get_colors_to_cycle_rainbow8, get_colors_to_cycle_rainbowN
init_params()
rootsave = Path("output_plots/")

def make_video_parallel_static():
    experiments.graph_embedding_per_epoch = xp.trainer.graph_embedding_per_epoch
    experiments.node_embedding_per_epoch = xp.trainer.node_embedding_per_epoch
    experiments.dataset = xp.trainer.dataset
    experiments.loss_list = xp.trainer.test_loss_list
    experiments.exp_config = xp.trainer.config_class
    experiments.dataset_type = xp.trainer.gg.graphtype
    num_emb_neurons = xp.trainer.model.convs[-1].out_channels
    experiments.trainmode = xp.trainer.config_class.modo
    experiments.num_classes = xp.trainer.config_class.num_classes()
    experiments.embedding_dimension = num_emb_neurons
    #my_list = my_log_lista=list(range(20)) + list(range(20,100,4)) + list(range(100,500, 15)) + list(range(500, 5000, 40))
    nomefile = xp.make_video(skip=1, fromfiles=True, custom_list=True, seq_colors=True)
    return nomefile


env: CUBLAS_WORKSPACE_CONFIG=":4096:8"


In [2]:
config_file = "configurations/Final1.yml"
c = Config(config_file)
diz_trials = {'graph_dataset.ERmodel': [True, False], 'graph_dataset.confmodel': [True, False], 'graph_dataset.sbm': [True, False],
              'graph_dataset.Num_nodes': [245, [245]*10], 'graph_dataset.list_p': [[0.3, 0.2, 0.1, 0.05, 0.01], [[0.5, 0.01], [0.9, 0.05]]],
             'model.GCNneurons_per_layer': [[1, 32, 2],[1, 32, 1]], 
              'model.neurons_last_linear':[[16, 16, len(c.conf['graph_dataset']['list_exponents'])], [16, 16, len(c.conf['graph_dataset']['list_p'])]],
             'model.init_weights': ['xavier_normal', 'eye']}

xp = Experiments(config_file, diz_trials=diz_trials, rootsave=rootsave)#, config_class=c)

Controllare errori non di assert
Controllare errori non di assert
Controllare errori non di assert
Controllare errori non di assert
120 configurazioni saltate su 128, farò 8 training


In [3]:
nomifilesgif = []
k = 0
for c in xp.gc.configs:
    print(f'Run {k + 1}\t\t exp name: {c.unique_train_name}')
    # all_seeds()
    xp.trainer.reinit_conf(c)
    xp.just_train()
    embedding_class = xp.embedding()
    num_emb_neurons = xp.trainer.model.convs[-1].out_channels
    trainmode = xp.trainer.config_class.modo
    embedding_class.get_metrics(num_emb_neurons, trainmode)
    #my_list = my_log_lista=list(range(20)) + list(range(20,100,4)) + list(range(100,500, 10)) + list(range(500, xp.trainer.last_epoch, 20))
    nomefile = make_video_parallel_static()
    nomifilesgif.append(nomefile)
    k += 1

Run 1		 exp name: GraphType.ER_[0.3_0.2_0.1_0.05_0.01]_nodi245_grafi80_mode1_layers§1-32-2+16-16-5§_initwxavier_normal_lr0.01_freezedFalse
Generating dataset...
Mean connectivity for each node: 73.266 p=0.3
Mean connectivity for each node: 48.83 p=0.2
Mean connectivity for each node: 24.289 p=0.1
Mean connectivity for each node: 12.162 p=0.05
Mean connectivity for each node: 2.44 p=0.01
Dataset generated
Loading Dataset...


100%|███████████████████████████████████████████████████████████████████| 200/200 [00:07<00:00, 27.72it/s]


Tempo impiegato: 7.235293626785278


100%|███████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 11.09it/s]


test accuracy finale: 0.21
(101, 49000, 2)
Preparing data for make video...
79 è la len di id_node_emb
(2.98 s.)  ...data ready: make video...(14.88 s.)
GraphType.ER_[0.3_0.2_0.1_0.05_0.01]_nodi245_grafi80_mode1_layers§1-32-2+16-16-5§_initwxavier_normal_lr0.01_freezedFalse
Run 2		 exp name: GraphType.ER_[0.3_0.2_0.1_0.05_0.01]_nodi245_grafi80_mode1_layers§1-32-2+16-16-5§_initweye_lr0.01_freezedFalse
Generating dataset...
Mean connectivity for each node: 73.268 p=0.3
Mean connectivity for each node: 48.709 p=0.2
Mean connectivity for each node: 24.471 p=0.1
Mean connectivity for each node: 12.193 p=0.05
Mean connectivity for each node: 2.4 p=0.01
Dataset generated
Loading Dataset...


...

100%|███████████████████████████████████████████████████████████████████| 200/200 [00:07<00:00, 26.53it/s]


Tempo impiegato: 7.558684349060059


100%|███████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.39it/s]


test accuracy finale: 0.18
(101, 49000, 1)
Preparing data for make video...
79 è la len di id_node_emb
(6.87 s.)  ...data ready: make video...(21.95 s.)
GraphType.ER_[0.3_0.2_0.1_0.05_0.01]_nodi245_grafi80_mode1_layers§1-32-1+16-16-5§_initweye_lr0.01_freezedFalse
Run 5		 exp name: GraphType.SBM_[0.3_0.2_0.1_0.05_0.01]_nodi245_grafi80_mode1_layers§1-32-2+16-16-5§_initwxavier_normal_lr0.01_freezedFalse


AssertionError: Lo Stochastic Block Model richiede una matrice di probabilità

In [ ]:
np.array([0.5, 0.01]).ndim #, [0.9, 0.05]]).ndim